Now requires JHU_Each_State_Capture_and_Serialize_JHU_Latest_Data to be run first. Once ran a serialized version of the latest JHU data will be available locally thus avoiding multiple data fetches for the various related routines

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date, timedelta
import datetime
import matplotlib.dates as mdates
%matplotlib inline

Update end date below to the previous day's date

In [ ]:
yesterday = date.today() - timedelta(days=1)
yesterday.strftime('%m/%d/%Y')

In [ ]:
df_jhu_us_dict = pd.read_pickle(yesterday.strftime('%d_%b_%Y') + r'_JHU_data.pkl')

In [ ]:
x = pd.date_range(start='4/12/2020', end=yesterday)
#x = pd.date_range(start='7/12/2021', end='9/08/2021')

In [ ]:
date_list = []
for date_value in x.strftime('%m-%d-%Y'):
    date_list.append(date_value)
dt_date_list = pd.to_datetime(pd.Series(date_list))

In [ ]:
list(df_jhu_us_dict.keys())[0]

In [ ]:
State_list =list(df_jhu_us_dict[list(df_jhu_us_dict.keys())[0]].Province_State)

In [ ]:
State_list.remove('Recovered')

In [ ]:
State_list

In [ ]:
def process_state_deaths(passed_state):
    print(passed_state)
    aaa = df_jhu_us_dict[date_list[0]].loc[df_jhu_us_dict[date_list[0]].Province_State == passed_state]
#    print(aaa)
    for d_value in date_list[1:]:
        aaa = aaa.append(df_jhu_us_dict[d_value].loc[df_jhu_us_dict[d_value].Province_State == passed_state])
#    return(xxx)
    bbb = aaa.loc[:,['dt_date','Deaths']]
    ccc = bbb.set_index('dt_date')
    ddd = ccc.diff(periods = 1)
    ddd_7day_moving = ddd.rolling(7).mean().shift()
    return(ddd,passed_state,ddd_7day_moving)

In [ ]:
def chart_daily_deaths(passed_df, passed_state_name, ddd_7day_moving):
    months = mdates.MonthLocator()
    days = mdates.DayLocator()
    timeFmt = mdates.DateFormatter('%b %Y')
    fig, ax = plt.subplots()
    fig.set_size_inches(18,10)
    ax.xaxis.set_major_locator(months)
    ax.xaxis.set_major_formatter(timeFmt)
#ax.xaxis.set_minor_locator(months)
    plt.title('Johns Hopkins University ***' + passed_state_name + '*** Daily COVID Deaths')
    plt.xlabel('Date',color='black')
    plt.ylabel('New COVID Deaths',color='black')
    plt.bar(passed_df.index,passed_df.Deaths)
    plt.plot(ddd_7day_moving,'ro-')
    plt.legend(['7 Day Rolling Average','Daily Deaths'])
    fig.savefig('JHUDaily '+ passed_state_name + ' Deaths.png')
plt.close()

In [ ]:
for x in State_list:
    return_deaths = process_state_deaths(x)
    chart_daily_deaths(return_deaths[0],return_deaths[1],return_deaths[2])

In [ ]:
for x in ['California','Oregon','Alabama','Florida','Georgia', 'Mississippi','Alaska','Ohio','Idaho','Maine','Connecticut','Vermont','Texas','Arkansas','North Carolina']:
    return_deaths = process_state_deaths(x)
    chart_daily_deaths(return_deaths[0],return_deaths[1],return_deaths[2])